<a href="https://colab.research.google.com/github/quocanh34/IELTS-Data-Analysis/blob/main/Scrapping_and_cleaning_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
#open site url

url = "https://www.ielts.org/for-researchers/test-statistics/demographic-data"
html = urlopen(url)

In [ ]:
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [ ]:
title = soup.title
print(title)

<title>IELTS   Demographic data 2019</title>


In [ ]:
text = soup.get_text()


In [ ]:
# all_links = soup.find_all('a')
# for link in all_links:
#   print(link.get('href'))
rows = soup.find_all('tr')
# print(rows)
for row in rows:
  row_td = row.find_all('td')
print(row_td)
print(type(row_td))


[<td style="text-align: left;">Zimbabwe</td>, <td style="text-align: left;">0%</td>, <td style="text-align: left;">0%</td>, <td style="text-align: left;">0%</td>, <td style="text-align: left;">0%</td>, <td style="text-align: left;">3%</td>, <td style="text-align: left;">7%</td>, <td style="text-align: left;">16%</td>, <td style="text-align: left;">22%</td>, <td style="text-align: left;">25%</td>, <td style="text-align: left;">20%</td>, <td style="text-align: left;">6%</td>, <td style="text-align: left;">0%</td>]
<class 'bs4.element.ResultSet'>


In [ ]:
#removing HTML tags 
list_rows = []
for row in rows:
  row_td = row.find_all('td')
  str_cells= str(row_td)
  cleantext = BeautifulSoup(str_cells, 'lxml').get_text()
  list_rows.append(cleantext)

print(list_rows)




['[Reason, Below 4, \xa04, 4.5, 5, 5.5, 6, 6.5, 7, \xa07.5, \xa08, \xa08.5, 9]', '[\xa0For employment\n\xa0\n, \xa01%, \xa01%, 2%, 5%, 11%, 18%, 22%, 21%, 12%, 5%, 1%, \xa00%]', '[\xa0For higher education extended course (three months or more)\n\xa0\n, \xa01%, \xa01%, 4%, 11%, 21%, 24%, 17%, \xa011%, \xa06%, \xa03%\xa0, \xa01%, \xa00%]', '[\xa0For higher education short course (three months or less), \xa01%, \xa02%, 5%, 13%, 26%, 26%, \xa015%, \xa07 %, 4%\xa0, \xa02%, \xa00%, \xa00%]', '[\xa0For immigration, \xa01%, \xa01%, 2%, 7%, 15%, 21%, \xa018%, \xa015%, 11 %, \xa07%, \xa02%, \xa00%]', '[\xa0For other education purpose, \xa01%, \xa02%, 5%, 11%, 19%, 22%, \xa017%\xa0, \xa012%, \xa08%, \xa03%\xa0, \xa00%, \xa00%]', '[\xa0For personal reasons, \xa01%, \xa02%, 6%, 13%, 21%, 22%, \xa017%, \xa011%, \xa06%, \xa02%, \xa00%, \xa00%]', '[\xa0For professional registration (NOT medical), \xa00%, \xa01%, 2%, 4%, 7%, 12%, \xa017%, \xa021%, \xa020%, \xa013%, \xa03%, \xa00%]', '[\xa0For registrat

In [ ]:
#removing commas, square brackets, and selecting dataframe subset

df = pd.DataFrame(list_rows)
selected_frame = df.iloc[0:12, 0:13]

selected_frame = selected_frame[0].str.split(',', expand = True)
# print(selected_frame)
selected_frame[0] = selected_frame[0].str.strip('[')
selected_frame[12] = selected_frame[12].str.strip(']')
selected_frame[0] = selected_frame[0].replace('\n','', regex=True)
selected_frame[12] = selected_frame[12].replace('\n','', regex=True)

for i in range(1,13):
  selected_frame[i] = selected_frame[i].str.strip()
  selected_frame[i] = selected_frame[i].replace('\xa0', '')
  
print(selected_frame)



                                                   0        1   ...   11  12
0                                              Reason  Below 4  ...  8.5   9
1                                     For employment        1%  ...   1%  0%
2    For higher education extended course (three m...       1%  ...   1%  0%
3    For higher education short course (three mont...       1%  ...   0%  0%
4                                     For immigration       1%  ...   2%  0%
5                         For other education purpose       1%  ...   0%  0%
6                                For personal reasons       1%  ...   0%  0%
7         For professional registration (NOT medical)       0%  ...   3%  0%
8                       For registration as a dentist       0%  ...   2%  0%
9                        For registration as a doctor       0%  ...   5%  0%
10      For registration as a nurse (including CGFNS)       0%  ...   1%  0%
11                                           (Other)        1%  ...   1%  0%

In [ ]:
new_header = selected_frame.iloc[0] #grab the first row for the header
selected_frame = selected_frame[1:] #take the data less the header row
selected_frame.columns = new_header #set the header row as the df header
selected_frame

,Reason,Below 4,4,4.5,5,5.5,6,6.5,7,7.5,8,8.5,9
1,For employment,1%,1%,2%,5%,11%,18%,22%,21%,12%,5%,1%,0%
2,For higher education extended course (three m...,1%,1%,4%,11%,21%,24%,17%,11%,6%,3%,1%,0%
3,For higher education short course (three mont...,1%,2%,5%,13%,26%,26%,15%,7 %,4%,2%,0%,0%
4,For immigration,1%,1%,2%,7%,15%,21%,18%,15%,11 %,7%,2%,0%
5,For other education purpose,1%,2%,5%,11%,19%,22%,17%,12%,8%,3%,0%,0%
6,For personal reasons,1%,2%,6%,13%,21%,22%,17%,11%,6%,2%,0%,0%
7,For professional registration (NOT medical),0%,1%,2%,4%,7%,12%,17%,21%,20%,13%,3%,0%
8,For registration as a dentist,0%,1%,4%,9%,14%,13%,15%,18%,14%,8%,2%,0%
9,For registration as a doctor,0%,0%,1%,2%,4%,7%,12%,21%,27%,21%,5%,0%
10,For registration as a nurse (including CGFNS),0%,0 %,1%,2%,6%,17%,28%,27%,15%,4%,1%,0%


In [ ]:
#exporting to excel file and download

from google.colab import files

# selected_frame.to_excel('IELTS_data_analysis.xlsx', encoding = 'utf-8-sig') 
# files.download('IELTS_data_analysis.xlsx')
selected_frame.to_csv('IELTS_data_analysis.csv', encoding = 'utf-8-sig') 
files.download('IELTS_data_analysis.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>